# Imports

In [ ]:
from google.colab import drive
import os
from glob import glob
from tqdm.notebook import tqdm
from shutil import copy, rmtree
import cv2
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from time import time

In [ ]:
from IPython.display import clear_output
!pip install --upgrade kaggle
clear_output()
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [ ]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 97% 788M/814M [00:02<00:00, 316MB/s]
100% 814M/814M [00:02<00:00, 306MB/s]


In [ ]:
!unzip -qx /content/dogs-vs-cats-redux-kernels-edition.zip

In [ ]:
!unzip -qx /content/train.zip

# Prepared Data

In [ ]:
all_dogs = glob('/content/train/*cat*')
all_cats = glob('/content/train/*dog*')

print('Count of all dogs: ' ,len(all_dogs))
print('Count of all cats: ' ,len(all_cats))

Count of all dogs:  12500
Count of all cats:  12500


In [ ]:
all_images=glob('/content/train/*')
len(all_images)

25000

In [ ]:
t=0
for img in all_images:
  img_read = cv2.imread(img, cv2.IMREAD_UNCHANGED)
  if img_read is None:
    t+=1
print(t)

0


In [ ]:
train_dir = '/content/cats-v-dogs/training/'
test_dir = '/content/cats-v-dogs/testing/'
to_create = [train_dir,test_dir]
for dir in to_create:
  os.makedirs(dir,exist_ok=True)

In [ ]:
training_images, testing_images = train_test_split(all_images,test_size=0.1,random_state=101)
print(len(training_images))
print(len(testing_images))

22500
2500


In [ ]:
testing_images[0]

'/content/train/dog.6774.jpg'

In [ ]:
print(train_dir)
for name in tqdm(training_images):
     copy(name , os.path.join(train_dir , name.split('/')[-1]))
print(test_dir)
for name in tqdm(testing_images):
      copy(name , os.path.join(test_dir , name.split('/')[-1]))

/content/cats-v-dogs/training/


  0%|          | 0/22500 [00:00<?, ?it/s]

/content/cats-v-dogs/testing/


  0%|          | 0/2500 [00:00<?, ?it/s]

# Create Dataset

In [ ]:
# Dataset
class CatsDogs(Dataset):
  def __init__(self, directory = '/content/cats-v-dogs/testing' , transform = None):
    super().__init__()
    self.names = glob(directory+'/*.jpg')[:]
    self.labels = []
    self.transform = transform
    
    for name in self.names: 
      if 'cat.' in name:
        self.labels.append(0)
      else:
        self.labels.append(1)
    self.labels = torch.tensor(self.labels, dtype = torch.float32)
    self.labels = torch.unsqueeze(self.labels, 1)

  def __len__(self):
    return len(self.names)
  
  def __getitem__(self, idx):
    x = Image.open(self.names[idx])
    x = self.transform(x) 
    y = self.labels[idx]
    return x,y

In [ ]:
train_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Resize((160,160)),
                                       transforms.Normalize(mean=([0.485, 0.456, 0.406]),std=([0.229, 0.224, 0.225])),
                                       transforms.RandomRotation(2.8),
                                       ])

test_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((160,160)),
                                      transforms.Normalize(mean=([0.485, 0.456, 0.406]),std=([0.229, 0.224, 0.225]))
                                      ])

In [ ]:
train_dataset = CatsDogs(directory='/content/cats-v-dogs/training',transform=train_transforms)
test_dataset = CatsDogs(directory='/content/cats-v-dogs/testing',transform=test_transforms)
train_loader = DataLoader(train_dataset,num_workers=2,batch_size=64)
test_loader = DataLoader(test_dataset,num_workers=2,batch_size=64)

In [ ]:
train_loader_it = iter(train_loader)
test_loader_it=iter(test_loader)

In [ ]:
print(next(train_loader_it)[0].shape)
print(next(train_loader_it)[1].shape)

torch.Size([64, 3, 160, 160])
torch.Size([64, 1])


#Create Model

In [ ]:
class My_Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3)
    self.conv2=nn.Conv2d(in_channels=16,out_channels=20,kernel_size=3)

    self.conv3=nn.Conv2d(in_channels=20,out_channels=28,kernel_size=3)
    self.conv4=nn.Conv2d(in_channels=28,out_channels=32,kernel_size=3)

    self.conv5=nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3)
    self.conv6=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)

    self.relu=nn.ReLU()
    self.maxpool=nn.MaxPool2d(2)
    self.flatten=nn.Flatten()
    self.sigmoid=nn.Sigmoid()
    self.dropout=nn.Dropout2d(0.2)

    self.fc1=nn.Linear(in_features=16384,out_features=32)
    self.fc2=nn.Linear(in_features=32,out_features=1)


  def forward(self, input):
    # print(input.shape)
    y=self.conv1(input)
    y=self.relu(y)
    y=self.conv2(y)
    y=self.relu(y)
    y=self.maxpool(y)
    y=self.dropout(y)
    # print(y.shape)

    y=self.conv3(y)
    y=self.relu(y)
    y=self.conv4(y)
    y=self.relu(y)
    y=self.maxpool(y)
    y=self.dropout(y)
    # print(y.shape)

    y=self.conv5(y)
    y=self.relu(y)
    y=self.conv6(y)
    y=self.relu(y)
    y=self.maxpool(y)
    y=self.dropout(y)
    # print(y.shape)

    y=self.flatten(y)
    # print(y.shape)

    y=self.fc1(y)
    y=self.relu(y)
    y=self.fc2(y)
    # y=self.sigmoid(y)
    # print(y.shape)
    # print(y)

    return(y)

In [ ]:
model=My_Classifier().forward(next(train_loader_it)[0])
x = torch.randn(1, 3, 160, 160)
My_Classifier().forward(x)


tensor([[0.0888]], grad_fn=<AddmmBackward0>)

# Train Model

In [ ]:
device = ('cuda' if torch.cuda.is_available else 'CPU')
mymodel = My_Classifier().to(device)
print(device)

cuda


In [ ]:
sigmoid=nn.Sigmoid()
def binary_acc(y_pred, y_test):
    y_p=torch.tensor([1 if sigmoid(p)>=0.5 else 0 for p in y_pred]).to(device)
    correct_results_sum=(y_p==y_test).sum().float(
    return correct_results_sum.item()

In [ ]:
epochs = 30
criterion = nn.BCEWithLogitsLoss()
opt = torch.optim.Adam(mymodel.parameters())
losses = [] 
val_losses = []
train_accuracy = []
val_accuracy = []

In [ ]:
for epoch in range(epochs):
  ##train model##
  tic=time()
  mymodel.train()
  batch_loss=[]
  batch_acc=0
  for batch in train_loader:
    x=batch[0].to(device)
    y=batch[1].to(device)
    # print(x.shape)
    y_pred=mymodel.forward(x)
    opt.zero_grad()
    loss=criterion(y_pred,y)
    loss.backward()
    opt.step()
    batch_loss.append(loss.item())
    batch_acc += binary_acc(y_pred,y)

  losses.append(np.mean(batch_loss)) 

  train_accuracy.append((batch_acc/len(train_dataset)))
  ##eval model##
  mymodel.eval()
  batch_loss=[]
  batch_acc=0
  for batch in test_loader:
    x=batch[0].to(device)
    y=batch[1].to(device)
    y_pred=mymodel.forward(x)
    with torch.no_grad():
       loss=criterion(y_pred,y)
       batch_loss.append(loss.item())
       batch_acc+=binary_acc(y_pred,y)
  
  val_losses.append(np.mean(batch_loss))
  val_accuracy.append((batch_acc/len(test_dataset)))
  toc=time()
  time_epoch=toc-tic
  print(f'Epoch {epoch}, loss : {losses[-1]}, accuracy : {train_accuracy[-1]}, val_loss : {val_losses[-1]}, val_accuracy : {val_accuracy[-1]}, time : {time_epoch}')

   


Epoch 0, loss : 0.6536174101585691, accuracy : 32.10355555555556, val_loss : 0.5736624121665954, val_accuracy : 32.2104, time : 117.36808514595032
Epoch 1, loss : 0.5683829522776332, accuracy : 32.129244444444446, val_loss : 0.4983791820704937, val_accuracy : 32.2896, time : 109.56127309799194
Epoch 2, loss : 0.5137773333117366, accuracy : 32.19573333333334, val_loss : 0.45881864577531817, val_accuracy : 32.3136, time : 111.69301867485046
Epoch 3, loss : 0.4692081180824475, accuracy : 32.21991111111111, val_loss : 0.42488099038600924, val_accuracy : 32.38, time : 110.78983449935913
Epoch 4, loss : 0.43080285885794595, accuracy : 32.25697777777778, val_loss : 0.4005224458873272, val_accuracy : 32.3, time : 111.06730437278748
Epoch 5, loss : 0.39398763616654003, accuracy : 32.26791111111111, val_loss : 0.37857292853295804, val_accuracy : 32.3528, time : 112.0330536365509
Epoch 6, loss : 0.3647665040652183, accuracy : 32.31413333333333, val_loss : 0.3715174371376634, val_accuracy : 32.326